<a href="https://colab.research.google.com/github/matheuspercario/TT007-AnaliseDadosPython/blob/master/exercicios/Aula5-POO/liarsdice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício - Liars Dice


Criado por: Matheus Bruder

## Instruções de jogo

No Liers’ dice, cada jogador recebe 5 dados. Em cada rodada os jogadores lançam os seus dados, mantendo suas faces escondidas dos demais jogadores.
Cada jogador então escolhe uma face (1 a 6) e da um palpite de quantas faces iguais a esta há na mesa.
O próximo a jogar tem três alternativas:
  * _Escolhe uma face e da um palpite, que deve ser necessariamente maior que o palpite anterior e passa para o próximo jogador._
  * _Chama o jogador anterior de mentiroso, neste caso abrem-se os dados e verifica-se o palpite corrente é verdadeiro. Se for o jogador atual perde um dado, senão o jogador anterior perde um dado._
  * _Concorda com o palpite do jogador anterior. Neste caso, abrem-se os dados e se o palpite estiver correto, o jogador anterior perde dois dados._

Ganha o último jogador com dados na mesa.

### Informações extras
* O jogo deve ser para um usuário contra até 4 jogadores controlados pelo computador;
* Use Objetos;
* Considere usar probabilidades para que um jogador do PC, de um palpite, chame de mentiroso ou concorde com o jogador anterior;
* Use diferentes probabilidade para diferentes jogadores para simular diferentes comportamentos;
* É possível utilizar-se de alguma heurística para melhorar os jogadores do PC?
* Quem fica mais tempo na mesa, um jogador com heurística ou um jogador aleatório?

---

# Preparação das classes

In [68]:
# Iniciar códigos aqui

In [76]:
import random
import time

from os import system
from sys import exit

In [77]:
class Player:


    def __init__(self, name, dice, num_die=5):
        self._name = name
        self._dice = dice
        self._num_die = num_die


    @property
    def name(self):
        """Return the name."""
        return self._name


    @name.setter
    def name(self, name):
        """Set the name."""
        name.strip().capitalize()
        if len(name) > 25:
            raise ValueError(f'Name must be less than 25 characters')

        self._name = name


    @property
    def dice(self):
        """Return the number of die."""
        return self._dice
    

    @property
    def num_die(self):
        """Return the number of die."""
        return self._num_die


    @num_die.setter
    def num_die(self, num_die):
        """Set the number of dice."""
        self._num_die = num_die

    
    def roll_dice(self):
        self._dice = [0, 0, 0, 0, 0]
        for i in range(self.num_die):
            self.dice[i] = random.randrange(1, 7)
        # print(f'TEST -> DADOS:{self.dice}')


    def show_hand(self):
        string = ''
        for d in self.dice:
            if d == 0:
                continue
            string += f'[{d}] '
        print(f'{self.name}, your hand: {string}')

    
    def show_number_dice(self):
        string = ''
        for d in self.dice:
            if d == 0:
                continue
            string += f'[X] '
        print(f'{self.name} \t ({self.num_die}) dados: {string}')



    def lose_die(self):
        self._num_die -= 1


    def dice_count(self, value):
        t = 0
        for d in self.dice:
            if d == value:
                t += 1
        return t


    def bluff(self, l_players, who, old_bet):
        print(f'{self.name} acha que o jogador anterior está mentindo.\n')
        total = 0
        for player in l_players:
            num_dice = player.dice_count(old_bet[1])
            total += num_dice
            print(f'{player.name} possui {num_dice} ({old_bet[1]})')
        print(f'Existe um total de {total} ({old_bet[1]}) na mesa.\n')

        # Lógica
        if total >= old_bet[0]:
            self.lose_die()
            print(f'Errado! {self.name} perdeu um dado, agora possui {self.num_die} dados.')
        else:
            l_players[(who - 1) % len(l_players)].lose_die()
            print(f'Boa, {self.name}! O jogador anterior perdeu um dado.')


    def agree(self, l_players, old_bet):
        print(f'{self.name} acha que o jogador anterior está falando a verdade.\n')
        total = 0
        for player in l_players:
            num_dice = player.dice_count(old_bet[1])
            total += num_dice
            print(f'{player.name} possui {num_dice} ({old_bet[1]})')
        print(f'Existe um total de {total} ({old_bet[1]}) na mesa.\n')

        # Lógica
        if total == old_bet[0]:
            print(f'Boa, {self.name}! Todo mundo perdeu um dado.')
            for player in l_players:
                if player.name != self.name:
                    player.lose_die()
                    print(f'{player.name} tem agora apenas {player.num_die} dados.')
        else:
            self.lose_die()
            print(f'Errado! {self.name} perdeu um dado, agora possui {self.num_die} dados.')


    def __repr__(self):
        """Return string representation for repr()."""
        return (f'(Nome={self.name}, Numero_dados={self.num_die}, Dados={self.dice})')
    

In [78]:
class NPC(Player):

    def __init__(self, name, dice, num_die=5):
        super().__init__(name, dice, num_die=5)


    def bet(self, old_bet, num_dice):
        new_bet = [0, 0]
        while True:
            b0 = random.randrange(1, num_dice + 1)
            b1 = random.randrange(1, 7)
            new_bet = [int(b0), int(b1)]
            if old_bet[0] >= new_bet[0]:
                # print(f'{self.name}, aposta inválida, tente novamente.')
                continue
            break
        return new_bet
    

    def choose(self):
        r = random.randrange(0, 3)
        if r == 0:
            return 0
        elif r == 1:
            return 1
        elif r == 2:
            return 2
        pass
    

    def __repr__(self):
        """Return string representation for repr()."""
        return (self.__class__.__name__ + '\t-> ' + super().__repr__())

In [79]:
class Human(Player):

    def __init__(self, name, dice, num_die=5):
        super().__init__(name, dice, num_die=5)


    def bet(self, old_bet, num_dice):
        new_bet = [0, 0]
        while True:
            new_bet = input(f'{self.name} é sua vez, faça sua aposta! [Número de dados] [Valor do dado]\nÚltima aposta [{old_bet[0]}] [{old_bet[1]}]\n: ').split()
            if len(new_bet) == 2 and new_bet[0].isdigit() and new_bet[1].isdigit():
                if old_bet[0] >= int(new_bet[0]) or int(new_bet[0]) > num_dice or int(new_bet[1]) > 6 or int(new_bet[1]) < 1:
                    print('Aposta inválida, tente novamente.')
                    continue
                else:
                    new_bet = [int(new_bet[0]), int(new_bet[1])]
                    break
        return new_bet


    def choose(self):
        print(
        'Você gostaria de [a]postar, chamar o jogador anterior de [m]entiroso, ou [c]oncordar com o palpite do jogador anterior?')
        while True:
            res = input(': ').strip().lower()
            print()
            if res == 'a':
                return 0
            if res == 'm':
                return 1
            if res == 'c':
                return 2
            print('Escolha inválida. Escolha entre as letras: a, m, ou c.')


    def __repr__(self):
        """Return string representation for repr()."""
        return (self.__class__.__name__ + '\t-> ' + super().__repr__())

# Funções & dados auxiliares

# Jogo começa aqui

In [ ]:
NAMES = 'Josie Henry Adam Carl Aidan Ryan Jack Conrad Gary Ava Susan Lily Oscar Mason Roy Claire'.split()

player_name = input('Qual é seu nome?\n: ').strip().capitalize()

while True:
    num_players = int(input('\nQuantos jogadores?\n: '))
    if 2 <= num_players <= 5:
        break

print('\n\n\n')

players = []
dice = [0, 0, 0, 0, 0]

for n in range(num_players - 1):
    name = NAMES[random.randint(0, len(NAMES)-1)]
    NAMES.remove(name)
    players.append(NPC(name=name, dice=dice))

players.append(Human(name=player_name, dice=dice))

gameContinues = True
nextPlayer = random.randrange(0, len(players))  # definir quem começa
#print(f'nextPlayer -> {nextPlayer}')

# Inicio de jogo
while gameContinues:
    n_dice = 0
    # definir dados da rodada e somar total
    print()
    for p in players:
        p.roll_dice()
        p.show_number_dice()
        n_dice += p.num_die
    print()
        
    current_player = players[nextPlayer]
    current_bet = [0, 0]

    if current_player.__class__.__name__ == 'Human':
        print()
        current_player.show_hand()
        print(f'Existem {n_dice - current_player.num_die} outros dados na mesa.')

    print(f'\n{current_player.name} começa. Por favor, faça a primeira aposta.')

    current_bet = current_player.bet(current_bet, n_dice)
    print(f'{current_player.name} apostou que existem {str(current_bet[0])} dados com valor [{str(current_bet[1])}] na mesa.\n')

    
    # Continuando os rounds
    status = True
    while status:
        nextPlayer = (nextPlayer + 1) % len(players)
        #print(f'nextPlayer -> {nextPlayer}')

        current_player = players[nextPlayer]

        # Mostrando opções ao jogador
        if current_player.__class__.__name__ == 'Human':
            print()
            current_player.show_hand()

            n_dice = 0
            # Contar numero de dados em jogo
            for p in players:
                n_dice += p.num_die
            print(f'Existem outros {n_dice - current_player.num_die} dados na mesa.')

        # Tomando decisão (apostar, duvidar, concordar)
        c = current_player.choose()

        print(f'Sua vez {current_player.name}...')
         # Determinando escolhas
        if c == 0:
            current_bet = current_player.bet(current_bet, n_dice)
            print(f'{current_player.name} apostou que existem {str(current_bet[0])} dados com valor [{str(current_bet[1])}] na mesa.\n')

        elif c == 1:
            current_player.bluff(players, nextPlayer, current_bet)
            status = False

        elif c == 2:
            current_player.agree(players, current_bet)
            status = False

    # Remover jogador da partida
    for p in players:
        if p.num_die == 0:
            print(f'\nGame over para {p.name}, até a próxima...')
            players.pop(players.index(p))

    nextPlayer = (nextPlayer - 1) % len(players)
    #print(f'nextPlayer -> {nextPlayer}')

    if len(players) == 1:
        gameContinues = False
        print(f'\n\nPARABÉNS! {current_player.name} ganhou o jogo.')

Qual é seu nome?
: Mak

Quantos jogadores?
: 2





Aidan 	 (5) dados: [X] [X] [X] [X] [X] 
Mak 	 (5) dados: [X] [X] [X] [X] [X] 


Aidan começa. Por favor, faça a primeira aposta.
Aidan apostou que existem 5 dados com valor [6] na mesa.


Mak, your hand: [3] [3] [6] [2] [6] 
Existem outros 5 dados na mesa.
Você gostaria de [a]postar, chamar o jogador anterior de [m]entiroso, ou [c]oncordar com o palpite do jogador anterior?
: a

Sua vez Mak...
Mak é sua vez, faça sua aposta! [Número de dados] [Valor do dado]
Última aposta [5] [6]
: 6 6
Mak apostou que existem 6 dados com valor [6] na mesa.

Sua vez Aidan...
Aidan apostou que existem 8 dados com valor [3] na mesa.


Mak, your hand: [3] [3] [6] [2] [6] 
Existem outros 5 dados na mesa.
Você gostaria de [a]postar, chamar o jogador anterior de [m]entiroso, ou [c]oncordar com o palpite do jogador anterior?
: m

Sua vez Mak...
Mak acha que o jogador anterior está mentindo.

Aidan possui 0 (3)
Mak possui 2 (3)
Existe um total de 2 (3) na mesa

# Questionamentos & Respostas
 - **É possível utilizar-se de alguma heurística para melhorar os jogadores do PC?**

   Com certeza deve ser possível implemetar alguma heurística para melhorar os NPCs (jogadores do controlados pelo computador), porém, dado o curto prazo de 1 semana não consegui implementar tal funcionalidade. 

   Contudo, provavelmente, acredito que deve ser algum calculo relacionados a probabilidade, pois, em minha implementação as vezes os NPCs realizam escolhas muito absurdas, por exemplo, existem 15 dados na mesa e eles apostam que os 15 dados possuem o mesmo valor. Isso faz com que fique fácil para o *player (human)* dizer que esse escolha provavelmente é uma mentira.

   A probabilidade dita anteriormente, certamente deve levar em conta as apostas dos jogadores anteriores. Isto é, se um jogador apostou que existem 4 dados com valor 6, certamente ele possui algum dado com valor 6, então, a partir dessa aposta dá para ir retirando probabilidades e possuir uma chance maior de acerto.
 


 - **Quem fica mais tempo na mesa, um jogador com heurística ou um jogador aleatório?**

   Certamente um jogador (NPC) com heurística atrelada a probabilidade ficará mais tempo na mesa, pois, suas apostas serão mais coerentes com a realidade, dando-o mais chances de acerto e, consequentemente, deixará o jogo mais interessante.

   Pense... Existem 12 dados na mesa, é mais provavel que existem 12 dados com valor X ou 3, 4 ou 5 dados com valor X?! 


